## Load data

In [1]:
import h5py as h5
import numpy as np
from matplotlib import pyplot as plt

In [58]:
path_1 = 'C:/data/datasets/eqt/eqt_sakhalin_bandpass/eqt_seisan_1.h5'
path_2 = 'C:/data/datasets/eqt/eqt_sakhalin_bandpass/eqt_seisan_3.h5'

## Transform and merge data

Model input data excpected in shape (None, 6000, 3) while we save data in shape (None, 3, 6000)

In [59]:
def convert_X(X):
    
    r_shape = (X.shape[0], X.shape[2], X.shape[1])
    r_X = np.zeros(r_shape)
    
    for i in range(r_shape[0]):
        for j in range(r_shape[2]):
            r_X[i, :, j] = X[i, j, :]
            
    return r_X

In [60]:
def to_N_E_Z(X):
    """
    From Z, N, E to N, E, Z
    """
    r_X = np.zeros(X.shape)
    for i in range(X.shape[0]):
        r_X[i, :, 0] = X[i, :, 1]
        r_X[i, :, 1] = X[i, :, 2]
        r_X[i, :, 2] = X[i, :, 0]
        
    return r_X

In [61]:
def merge_wave(x1, x2):
    
    shape = list(x1.shape)
    shape[0] = x1.shape[0] + x2.shape[0]
    shape = tuple(shape)
    
    r_x = np.zeros(shape)
    
    r_x[:x1.shape[0]] = x1
    r_x[x1.shape[0]:] = x2
    
    return r_x

In [62]:
def merge_str(x1, x2):
    
    return list(x1) + list(x2)

In [63]:
def load_data(*paths, convert = True, to_n_e_z = True):
    
    Xs = []
    Ps = []
    Ss = []
    
    for path in paths:
        with h5.File(path_1, 'r') as file:
            
            X = np.array(file['X'])
            X = convert_X(X)
            Xs.append(X)
            
            Ps.append(np.array(file['P']))
            Ss.append(np.array(file['S']))

    X = Xs[0]
    P = Ps[0]
    S = Ss[0]
    for i in range(1, len(Xs)):

        X = merge_wave(X, Xs[i])
        P = merge_str(P, Ps[i])
        S = merge_str(S, Ss[i])

    X = to_N_E_Z(X)
    
    return X, P, S

In [64]:
X, P, S = load_data(path_1, path_2)

In [9]:
X.shape, len(P), len(S)

((2948, 6000, 3), 2948, 2948)

## Load EQT

In [10]:
model_path = 'C:/dev/EQTransformer/ModelsAndSampleData/EqT_model.h5'

In [11]:
from tensorflow.keras.models import load_model

In [12]:
# Modifying sys.path to be able to load project packages
import sys
sys.path.append('C:/dev/EQTransformer/EQTransformer/core')

In [13]:
from EqT_utils import f1, SeqSelfAttention, FeedForward, LayerNormalization

In [14]:
model = load_model(model_path, 
                   custom_objects={'SeqSelfAttention': SeqSelfAttention, 
                                   'FeedForward': FeedForward,
                                   'LayerNormalization': LayerNormalization, 
                                   'f1': f1                                                                            
                                    })   

## Predict

In [15]:
results = model.predict(X)

np_results = np.array(results)
np_results = np_results.reshape(np_results.shape[:3])

In [16]:
# result label, input index, sample
# Where 0 - detection label, 1 - p label, 2 - s label
np_results.shape

(3, 2948, 6000)

In [17]:
def plot_eqt_wave_prediction(X, P, S, i_input, predictions, i_prediction = -1):
    
    if i_prediction == -1:
        i_prediction = i_input
    
    p_positions = [int(x) for x in P[i_input].split(';') if len(x)]
    s_positions = [int(x) for x in S[i_input].split(';') if len(x)]

    data = X[i_input]
    
    fig = plt.figure(figsize = (7, 6), dpi = 160)
    axes = fig.subplots(6, 1, sharex = True)
    
    pred = predictions[:, i_prediction, :]
    
    # Plot waveforms
    for i in range(data.shape[1]):
        axes[i].plot(data[:, i], linewidth = 0.5, color = '#000')

    # Plot predictions
    for i in range(pred.shape[0]):
        axes[3 + i].plot(pred[i, :], 'g', linewidth = 0.5)
        
    # Mark seismic wave arrivals
    for x in p_positions:
        for i in range(data.shape[1]):
            axes[i].plot(x, data[x, i], 'g^', markersize = 7)

    for x in s_positions:
        for i in range(data.shape[1]):
            axes[i].plot(x, data[x, i], 'b^', markersize = 7)

In [18]:
from scipy.signal import find_peaks

In [19]:
def prediction_validate(X, P, S, predictions, i,
                       threshold = 0.2, min_distance = 200):
    
    p_positions = [int(x) for x in P[i].split(';') if len(x)]
    s_positions = [int(x) for x in S[i].split(';') if len(x)]

    data = X[i]
    pred = predictions[:, i, :]
    
    result = {
        'p':
        {
            'TP': 0,
            'TN': 0,
            'FP': 0,
            'FN': 0,
            'total': 0,
        },
        's':
        {
            'TP': 0,
            'TN': 0,
            'FP': 0,
            'FN': 0,
            'total': 0,
        },
    }
    
    # Get P predictions
    p_peaks_init = find_peaks(pred[1, :], distance = min_distance, height=[threshold, 100.])[0]
    
    p_peaks = [x for x in p_peaks_init if x > min_distance and x < pred.shape[1] - min_distance]
    
    # Check True Negatives
    if not len(p_peaks) and not len(p_positions):
        result['p']['TN'] += 1
    
    # Check True Positives and False Positives
    for peak_position in p_peaks:
        
        found = False
        for real_position in p_positions:
            if abs(real_position - peak_position) < min_distance:
                result['p']['TP'] += 1
                found = True
                break
        if not found:
            result['p']['FP'] += 1
            
    # Check False Negatives
    for real_position in p_positions:
        
        found = False
        for peak_position in p_peaks:
            if abs(real_position - peak_position) < min_distance:
                found = True
                break
        if not found:
            result['p']['FN'] += 1
    
    # Get S predictions
    s_peaks_init = find_peaks(pred[2, :], distance = min_distance, height=[threshold, 100.])[0]
    
    s_peaks = [x for x in s_peaks_init if x > min_distance and x < pred.shape[1] - min_distance]
    
    # Check True Negatives
    if not len(s_peaks) and not len(s_positions):
        result['s']['TN'] += 1
    
    # Check True Positives and False Positives
    for peak_position in s_peaks:
        
        found = False
        for real_position in s_positions:
            if abs(real_position - peak_position) < min_distance:
                result['s']['TP'] += 1
                found = True
                break
        if not found:
            result['s']['FP'] += 1
            
    # Check False Negatives
    for real_position in s_positions:
        
        found = False
        for peak_position in s_peaks:
            if abs(real_position - peak_position) < min_distance:
                found = True
                break
        if not found:
            result['s']['FN'] += 1

    result['p']['total'] = result['p']['TP'] + result['p']['FP'] + result['p']['TN'] + result['p']['FN']
    result['s']['total'] = result['s']['TP'] + result['s']['FP'] + result['s']['TN'] + result['s']['FN']
            
    return result

In [20]:
i = 110
plot_eqt_wave_prediction(X, P, S, i, np_results); prediction_validate(X, P, S, np_results, i)

{'p': {'TP': 1, 'TN': 0, 'FP': 0, 'FN': 1, 'total': 2},
 's': {'TP': 2, 'TN': 0, 'FP': 0, 'FN': 0, 'total': 2}}

### Find peaks with more than 2 wave arrivals

In [21]:
for i in range(X.shape[0]):
    
    p_positions = [int(x) for x in P[i].split(';') if len(x)]
    s_positions = [int(x) for x in S[i].split(';') if len(x)]
    
    if len(p_positions) + len(s_positions) > 2:
        print(i)

73
78
110
115
147
152
183
188
210
283
298
370
385
457
472
524
566
825
1072
1119
1169
1188
1245
1272
1325
1358
1372
1464
1549
1590
2034
2212
2221
2322
2357
2358
2394
2491
2649
2698


### Get total confusion matrix

In [22]:
def add_metrics(x1, x2):
    
    r_x = {}
    
    for key, data in x1.items():
        
        if type(data) is dict:
            r_x[key] = add_metrics(data, x2[key])
            
        else:
            r_x[key] = data + x2[key]
            
    return r_x

In [23]:
result = {
    'p':
    {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0,
        'total': 0,
    },
    's':
    {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0,
        'total': 0,
    },
}

for i in range(X.shape[0]):
    
    metrics = prediction_validate(X, P, S, np_results, i)
    result = add_metrics(result, metrics)
    
result

{'p': {'TP': 396, 'TN': 599, 'FP': 17, 'FN': 1980, 'total': 2992},
 's': {'TP': 356, 'TN': 153, 'FP': 130, 'FN': 2454, 'total': 3093}}

In [24]:
result['p']['TP'] + result['p']['FP'] + result['p']['TN'] + result['p']['FN'] == result['p']['total']

True

In [25]:
result['s']['TP'] + result['s']['FP'] + result['s']['TN'] + result['s']['FN'] == result['s']['total']

True

### P accuracy

In [26]:
(result['p']['TP'] + result['p']['TN']) / result['p']['total']

0.3325534759358289

### S accuracy

In [27]:
(result['s']['TP'] + result['s']['TN']) / result['s']['total']

0.16456514710636921

## Normalized test (std normalization)

In [28]:
def normalize(data, mode = 'std'):
    
    data -= np.mean(data, axis=0, keepdims=True)
    
    if mode == 'max':
        max_data = np.max(data, axis=0, keepdims=True)
        assert(max_data.shape[-1] == data.shape[-1])
        max_data[max_data == 0] = 1
        data /= max_data              

    elif mode == 'std':               
        std_data = np.std(data, axis=0, keepdims=True)
        assert(std_data.shape[-1] == data.shape[-1])
        std_data[std_data == 0] = 1
        data /= std_data
        
    return data

In [29]:
X, P, S = load_data(path_1, path_2)

In [30]:
for i in range(X.shape[0]):
    
    X[i] = normalize(X[i], mode = 'std')

In [31]:
results = model.predict(X)

np_results = np.array(results)
np_results = np_results.reshape(np_results.shape[:3])

In [32]:
result = {
    'p':
    {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0,
        'total': 0,
    },
    's':
    {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0,
        'total': 0,
    },
}

for i in range(X.shape[0]):
    
    metrics = prediction_validate(X, P, S, np_results, i, threshold = 0.2)
    result = add_metrics(result, metrics)
    
result

{'p': {'TP': 1620, 'TN': 545, 'FP': 87, 'FN': 743, 'total': 2995},
 's': {'TP': 1724, 'TN': 140, 'FP': 65, 'FN': 1074, 'total': 3003}}

### P accuracy

In [33]:
(result['p']['TP'] + result['p']['TN']) / result['p']['total']

0.7228714524207012

### S accuracy

In [34]:
(result['s']['TP'] + result['s']['TN']) / result['s']['total']

0.6207126207126207

## Normalized test (max normalization)

In [35]:
X, P, S = load_data(path_1, path_2)

In [36]:
for i in range(X.shape[0]):
    
    X[i] = normalize(X[i], mode = 'max')

In [37]:
results = model.predict(X)

np_results = np.array(results)
np_results = np_results.reshape(np_results.shape[:3])

In [38]:
result = {
    'p':
    {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0,
        'total': 0,
    },
    's':
    {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0,
        'total': 0,
    },
}

for i in range(X.shape[0]):
    
    metrics = prediction_validate(X, P, S, np_results, i)
    result = add_metrics(result, metrics)
    
result

{'p': {'TP': 240, 'TN': 580, 'FP': 20, 'FN': 2143, 'total': 2983},
 's': {'TP': 86, 'TN': 171, 'FP': 2, 'FN': 2733, 'total': 2992}}

### P accuracy

In [39]:
(result['p']['TP'] + result['p']['TN']) / result['p']['total']

0.27489104927924907

### S accuracy

In [40]:
(result['s']['TP'] + result['s']['TN']) / result['s']['total']

0.08589572192513369